In [ ]:
import os

from weco_datascience.reporting import get_recent_data

In [ ]:
df = get_recent_data(config=os.environ, n=100000, index="metrics-conversion-prod")

## We want to know whether concepts are helping researchers find what they are looking for. ##

Concepts pages started being offered to Collections users on 26 September. 
This analysis is to understand how effective they have been in helping
researchers using the Library catalogue in viewing works pages.

What period does the analysis cover? 

In [ ]:
print(df["@timestamp"].head(1))

In [ ]:
print(df["@timestamp"].tail(1))

## How frequently do people use concepts? ##

In [ ]:
concept_users=df.loc[(df["page.name"]=="concept")]
concept_users2=concept_users.drop_duplicates("anonymousId", keep="first")

print("Concepts users account for","{:.0%}".format(
    (concept_users["anonymousId"].nunique())/df["anonymousId"].nunique()),"of total unique users.")

## It looks like concepts usage is still pretty niche. ##

Maybe we can find out something about the people who are using them.

## But first, how about a function to make life easier? ##

-concepts page.name(concept)
<br>
-search page.name(works), page.query.query:*
<br>
-works page page.name(work), page.query.workId:*
<br>
-item view page.name(item)
<br>
-image view page.name(image)
<br>
-image modal properties.event(Open image modal)

In [ ]:
short=["anonymousId","@timestamp","page.name","page.query.query","page.query.workId","properties.event"]
df2=df[short]

df3=df2.assign(actionType=lambda x: "")

#replace empty cells 
df3.fillna("", inplace = True)
df4=df3.astype(str)
df4.head()


In [ ]:
def search(page_name, page_query_query):
    if ((page_name == "works") and (page_query_query != "")):
        return "search"

def concept(page_name):
    if page_name == "concept":
        return "concepts"

def works_page(page_name, page_query_workId):
    if ((page_name == "work") and (page_query_workId!="")):
        return "works page view"
    
def image_modal(properties_event):
    if properties_event == "Open image modal":
        return "image modal view"
    
def item_view(page_name):
    if page_name == "item":
        return "catalogue item view"
    
def image_view(page_name):
    if page_name == "image":
        return "images item view"

In [ ]:
for i in range(len(df3)):
    if df3.loc[i, "actionType"]=="":
        df3.loc[i, "actionType"]=concept(df3.loc[i, "page.name"]) 
        
df3.fillna("", inplace = True)

In [ ]:
for i in range(len(df3)):   
    if df3.loc[i, "actionType"]=="":
        df3.loc[i, "actionType"]=image_modal(df3.loc[i, "properties.event"]) 

df3.fillna("", inplace = True)

In [ ]:
for i in range(len(df3)):
    if df3.loc[i, "actionType"]=="":
        df3.loc[i, "actionType"]=works_page(df3.loc[i, "page.name"], df3.loc[i, "page.query.workId"]) 

df3.fillna("", inplace = True)

In [ ]:
for i in range(len(df3)):
    if df3.loc[i, "actionType"]=="":
        df3.loc[i, "actionType"]=search(df3.loc[i, "page.name"],df3.loc[i, "page.query.query"]) 

df3.fillna("", inplace = True)

In [ ]:
for i in range(len(df3)):
    if df3.loc[i, "actionType"]=="":
        df3.loc[i, "actionType"]=item_view(df3.loc[i, "page.name"]) 

df3.fillna("", inplace = True)

In [ ]:
for i in range(len(df3)):
    if df3.loc[i, "actionType"]=="":
        df3.loc[i, "actionType"]=image_view(df3.loc[i, "page.name"]) 

df3.fillna("", inplace = True)

##Evaluation of concept/non-concept user usage##

In [ ]:
clist=concept_users2["anonymousId"].tolist()
short=["anonymousId", "actionType"]
concept_usage=df3[short][df3["anonymousId"].isin (clist)]

non_concept_usage=df3[short][~df3["anonymousId"].isin (clist)]

In [ ]:
concept_usage["actionType"].value_counts()

In [ ]:
concept_usage["anonymousId"].nunique()

In [ ]:
non_concept_usage["actionType"].value_counts()

In [ ]:
non_concept_usage["anonymousId"].nunique()